In [ ]:
#Global parameters for no. of trips in each simulation , time steps between each time and size of dataset 

num_trips = 100
time_step = 2
datasize = 1000


In [ ]:
#enter your network file

network_file = "reduced.net.xml"

import xml.etree.ElementTree as ET

def get_edge_ids(net_file_path):
  """
  Reads a SUMO network file and returns a list of all edge IDs.

  Args:
    net_file_path: The path to the SUMO network file (.net.xml).

  Returns:
    A list of strings, where each string is an edge ID.
    Returns an empty list if the file is not found or if an error occurs.
  """
  try:
    tree = ET.parse(net_file_path)
    root = tree.getroot()
    edge_ids = [edge.get('id') for edge in root.findall('edge')]
    return edge_ids
  except FileNotFoundError:
    print(f"Error: File not found at {net_file_path}")
    return []
  except ET.ParseError:
    print(f"Error: Invalid XML format in {net_file_path}")
    return []
  except Exception as e:
    print(f"An unexpected error occurred: {e}")
    return []


# Example usage (replace with your actual file path):
edge_list = get_edge_ids(network_file)
print(f"done!")

# if edge_list:
#   print("Edge IDs in the network:")
#   for edge_id in edge_list:
#     print(edge_id)
# else:
#     print("Failed to retrieve edge IDs.")



In [ ]:

import xml.etree.ElementTree as ET

def get_nodes_from_edges(net_file_path):
    """
    Reads a SUMO network file and returns a list of all nodes from and to.

    Args:
      net_file_path: The path to the SUMO network file (.net.xml).

    Returns:
      A list of strings, where each string is a node ID.
      Returns an empty list if the file is not found or if an error occurs.
    """
    try:
        tree = ET.parse(net_file_path)
        root = tree.getroot()
        nodes = set()  # Use a set to avoid duplicate nodes
        for edge in root.findall('edge'):
            from_node = edge.get('from')
            to_node = edge.get('to')
            if from_node:
                nodes.add(from_node)
            if to_node:
                nodes.add(to_node)
        return list(nodes)  # Convert the set back to a list
    except FileNotFoundError:
        print(f"Error: File not found at {net_file_path}")
        return []
    except ET.ParseError:
        print(f"Error: Invalid XML format in {net_file_path}")
        return []
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return []

# Example usage:
node_list = get_nodes_from_edges(network_file)

if node_list:
    print("Nodes in the network:")
    for node_id in node_list:
        print(node_id)
else:
    print("Failed to retrieve nodes.")


In [ ]:

import networkx as nx

def create_graph_from_net_file(net_file_path):
    """
    Creates a graph from a SUMO network file.

    Args:
        net_file_path: The path to the SUMO network file (.net.xml).

    Returns:
        A NetworkX graph object representing the network, or None if an error occurs.
    """
    try:
        tree = ET.parse(net_file_path)
        root = tree.getroot()
        graph = nx.DiGraph()  # Use a directed graph for SUMO networks

        for edge in root.findall('edge'):
            from_node = edge.get('from')
            to_node = edge.get('to')
            edge_id = edge.get('id')
            if from_node and to_node and edge_id:
                graph.add_edge(from_node, to_node, id=edge_id) # Store edge ID as attribute
        return graph
    except FileNotFoundError:
        print(f"Error: File not found at {net_file_path}")
        return None
    except ET.ParseError:
        print(f"Error: Invalid XML format in {net_file_path}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Example usage:
network_graph = create_graph_from_net_file(network_file)

if network_graph:
    print("Graph created successfully!")
    # Now you can work with the network_graph object
    # For example, print some information about the graph:
    print(f"Number of nodes: {network_graph.number_of_nodes()}")
    print(f"Number of edges: {network_graph.number_of_edges()}")

    # Print edges with their IDs (attributes)
    for u, v, data in network_graph.edges(data=True):
      print(f"Edge from {u} to {v} with ID: {data['id']}")
else:
    print("Failed to create the graph.")


In [ ]:
# prompt: create a dual graph of the above

import xml.etree.ElementTree as ET
import networkx as nx

def create_dual_graph(net_file_path):
    """
    Creates a dual graph from a SUMO network file.

    Args:
        net_file_path: The path to the SUMO network file (.net.xml).

    Returns:
        A NetworkX graph object representing the dual graph, or None if an error occurs.
    """
    try:
        tree = ET.parse(net_file_path)
        root = tree.getroot()
        dual_graph = nx.Graph()  # Use an undirected graph for the dual

        # Assuming edges in the SUMO network represent connections between nodes
        for edge in root.findall('edge'):
            from_node = edge.get('from')
            to_node = edge.get('to')
            edge_id = edge.get('id')  # Use edge ID for dual graph nodes

            if from_node and to_node and edge_id:
                dual_graph.add_node(edge_id, from_node=from_node, to_node=to_node)

        # Connect nodes in the dual graph based on shared nodes in the original graph
        for edge1 in root.findall('edge'):
          for edge2 in root.findall('edge'):
            edge1_id = edge1.get('id')
            edge2_id = edge2.get('id')
            if edge1_id != edge2_id:
                if edge1.get('to') == edge2.get('from'):
                    dual_graph.add_edge(edge1_id, edge2_id)

        return dual_graph

    except FileNotFoundError:
        print(f"Error: File not found at {net_file_path}")
        return None
    except ET.ParseError:
        print(f"Error: Invalid XML format in {net_file_path}")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None


# Example usage
dual_graph = create_dual_graph(network_file)


if dual_graph:
  print("Dual graph created successfully!")
  print(f"Number of nodes in dual graph: {dual_graph.number_of_nodes()}")
  print(f"Number of edges in dual graph: {dual_graph.number_of_edges()}")
  for u, v in dual_graph.edges:
    print(f"Edge in dual graph: {u} - {v}")
else:
    print("Failed to create the dual graph.")


In [ ]:
import numpy as np

def create_adjacency_matrix(graph):
    """Creates an adjacency matrix for a given graph."""
    nodes = sorted(graph.nodes())
    n = len(nodes)
    adjacency_matrix = np.zeros((n, n), dtype=int)
    for u, v in graph.edges():
        u_index = nodes.index(u)
        v_index = nodes.index(v)
        adjacency_matrix[u_index, v_index] = 1
    return adjacency_matrix

def create_incidence_matrix(graph):
    """Creates an incidence matrix for a given graph."""
    nodes = sorted(graph.nodes())
    edges = list(graph.edges())  # Get edges as a list of tuples
    n = len(nodes)
    m = len(edges)

    incidence_matrix = np.zeros((n, m), dtype=int)
    for j, (u, v) in enumerate(edges):
        u_index = nodes.index(u)
        v_index = nodes.index(v)
        incidence_matrix[u_index, j] = 1
        incidence_matrix[v_index, j] = -1  # Or 1, depending on your convention
    return incidence_matrix


# Assuming 'dual_graph' is the graph you want to analyze
# (replace with 'network_graph' or any other graph variable if needed)

if dual_graph:
    adjacency_matrix = create_adjacency_matrix(dual_graph)
    incidence_matrix = create_incidence_matrix(dual_graph)

    print("Adjacency Matrix:")
    print(adjacency_matrix.shape)

    print("\nIncidence Matrix:")
    print(incidence_matrix.shape)
else:
    print("Dual graph not available. Please make sure the dual_graph is created successfully.")


In [ ]:
# creates TAZ file given network file

import xml.etree.ElementTree as ET

def create_taz_file(edge_list, output_file="taz.xml"):
    """
    Creates a TAZ file with each edge as a TAZ.
    """
    root = ET.Element("tazs")
    root.set("xmlns:xsi", "http://www.w3.org/2001/XMLSchema-instance")
    root.set("xsi:noNamespaceSchemaLocation", "http://sumo.dlr.de/xsd/taz_file.xsd")

    for i, edge_id in enumerate(edge_list):
      taz = ET.SubElement(root, "taz")
      taz.set("id", str(i+1))  # Assign TAZ IDs starting from 1
      taz.set("edges", edge_id)  # Use edge ID as TAZ shape
      # Add more attributes if needed, for example:
      # taz.set("x", "0")
      # taz.set("y", "0")


    tree = ET.ElementTree(root)
    ET.indent(tree, space="\t", level=0) # Add indentation
    tree.write(output_file, encoding="utf-8", xml_declaration=True)

# Example usage: (Assuming edge_list from previous code)
if edge_list:
  create_taz_file(edge_list)
  print(f"TAZ file 'taz.xml' created successfully.")
else:
  print(f"Failed to create TAZ file because edge list is empty.")


In [ ]:
#creates Trips file from TAZ files

import xml.etree.ElementTree as ET
import random
!pip install sumolib
import sumolib

net  = sumolib.net.readNet(network_file)
t=[] #place holder for no. of trips and time steps 

def generate_random_trips(taz_file, n, output_file="trips"):
    """Generates a random trips file with a specified number of trips."""
    new_trips = num_trips + int(random.uniform(-num_trips/2,num_trips/2)) #no. of trips is uniformly distributed with mean as num_trips and deviation of num_trips/2
    new_time = time_step + random.uniform(-time_step/2,time_step/2) #time_step is uniformly distributed with mean as time_step and deviation of time_step/2
    t.append([new_trips,new_time])

    try:
        tree = ET.parse(taz_file)
        root = tree.getroot()
        taz_ids = [[taz.get('id') , taz.get('edges')] for taz in root.findall('taz')]

        if not taz_ids:
            print(f"Error: No TAZs found in {taz_file}")
            return
    except FileNotFoundError:
        print(f"Error: TAZ file not found at {taz_file}")
        return
    except ET.ParseError:
        print(f"Error: Invalid XML format in {taz_file}")
        return

    trips_root = ET.Element("routes")
    trips_root.set("xmlns:xsi", "http://www.w3.org/2001/XMLSchema-instance")
    trips_root.set("xsi:noNamespaceSchemaLocation", "http://sumo.dlr.de/xsd/routes_file.xsd")

    for i in range(new_trips):
        # Choose random origin and destination TAZs
        origin = random.choice(taz_ids)
        destination = random.choice(taz_ids)

        shortestpath = net.getShortestPath(net.getEdge(origin[1]),net.getEdge(destination[1]))

        while shortestpath[0] is None or origin == destination:
            origin = random.choice(taz_ids)
            destination = random.choice(taz_ids)

            shortestpath = net.getShortestPath(net.getEdge(origin[1]),net.getEdge(destination[1]))

        # Ensure origin and destination are different and there is a valid route

        trip = ET.SubElement(trips_root, "trip")
        trip.set("id", f"{i+1}")
        trip.set("depart", str(i*new_time))  # Uniformly spaced departure times
        trip.set("from", origin[1])
        trip.set("to", destination[1])

        trip.set("fromTaz", origin[0])
        trip.set("toTaz", destination[0])
        trip.set("departLane", "free")
        trip.set("departSpeed", "max")


    tree = ET.ElementTree(trips_root)
    ET.indent(tree, space="\t", level=0)
    out = output_file+str(n)+".xml"
    tree.write(out, encoding="utf-8", xml_declaration=True)

    #print(f"Random trips file {out} created successfully.")


#iterative generation of trips file

for j in range(1,datasize):
   generate_random_trips("taz.xml", j)  # Pass the TAZ file path
   print(f"done!{j}")

In [ ]:
# capture input features : in_flux,out_flux and total_flux

import xml.etree.ElementTree as ET
import pandas as pd
import torch

def analyze_trips(trips_file, edge_list):
    """
    Analyzes a SUMO trips file and calculates edge coefficients.
    """

    try:
        tree = ET.parse(trips_file)
        root = tree.getroot()
        trips = root.findall('trip')
    except FileNotFoundError:
        print(f"Error: Trips file not found at {trips_file}")
        return None
    except ET.ParseError:
        print(f"Error: Invalid XML format in {trips_file}")
        return None

    edge_outflux = {edge+str("out"): 0 for edge in edge_list}
    edge_influx = {edge+str("in"): 0 for edge in edge_list}

    for trip in trips:
        from_edge = trip.get('from')
        to_edge = trip.get('to')

        if from_edge in edge_list:
            edge_outflux[from_edge+str("out")] += 1
        if to_edge in edge_list:
            edge_influx[to_edge+str("in")] -= 1
    edge_coefficients = {**edge_outflux,**edge_influx}

    return edge_coefficients
    


# Example usage:
edge_coefficients =[]
x = [] #total flux
z_in =[] #in flux
z_out = [] #out flux
for i in range(1,datasize):
  trips_files = "trips"+str(i)+".xml"
  edge_coefficients.append(analyze_trips(trips_files, edge_list))
  z_in.append([analyze_trips(trips_files,edge_list)[edge+str("in")] for edge in edge_list])
  z_out.append([analyze_trips(trips_files,edge_list)[edge+str("out")] for edge in edge_list])  
  x.append([analyze_trips(trips_files,edge_list)[edge+str("out")] for edge in edge_list]+[analyze_trips(trips_files,edge_list)[edge+str("in")] for edge in edge_list])
  print(f"done!{i}")

# Create a DataFrame from the edge coefficients
df = []
dfs = []
x = torch.tensor(x,dtype = torch.float)
z_in = torch.tensor(z_in,dtype = torch.float)
z_out = torch.tensor(z_out,dtype = torch.float)
print(x.shape)

for i in range(len(edge_coefficients)):
  df.append(pd.DataFrame(list(edge_coefficients[i].items()), columns=['Edge', 'Coefficient']))

# Display the DataFrame
# for i in range(99):
#    print(f"DataFrame {i+1}:")
#    print(df[i])

# print(edge_coefficients)

In [ ]:
# run the simulation in SuMO

taz_file = "taz.xml"
add_file = "addons.add.xml"
config_file = "sumo.sumocfg"
routes_file = "routes"

import os
import subprocess

def generate_random_simulations(j):
    trip_file = "trips"+str(j)+".xml"
    routes_file = "routes_WARB"+str(j)+".xml"
    config_file = "sumo"+str(j)+".sumocfg"

    with open(config_file, 'w') as f:
        f.write('<configuration>\n')
        f.write('    <input>\n')
        f.write(f'        <net-file value="{network_file}"/>\n')
        f.write(f'        <route-files value="{trip_file}"/>\n')
        f.write(f'        <additional-files value="{add_file},{taz_file}"/>\n')
        f.write('    </input>\n')
        f.write('    <output>\n')
        f.write(f'        <vehroute-output value="{routes_file}"/>\n')
        f.write(f'        <vehroute-output.route-length value="true"/>\n')
        f.write(f'        <vehroute-output.last-route value="true"/>\n')
        f.write(f'        <vehroute-output.write-unfinished value="true"/>\n')
        f.write(f'        <vehroute-output.exit-times value="true"/>\n')
        f.write(f'        <vehroute-output.skip-ptlines value="true"/>\n')
        f.write('    </output>\n')
    
        # Time configuration
        f.write('    <time>\n')
        f.write('        <begin value = "0"/>')
        f.write('        <step-length value="1" />\n')
        f.write('    </time>\n')

        # Routing parameters
        f.write('    <routing>\n')
        f.write('        <routing-algorithm value="astar" />\n')
        f.write('        <weights.random-factor value="1.8" />\n')
        f.write('        <weights.priority-factor value="0.5" />\n')
        f.write('        <device.rerouting.probability value="1" />\n')
        f.write('        <device.rerouting.period value="180" />\n')
        f.write('        <device.rerouting.pre-period value="0" />\n')
        f.write('        <device.rerouting.adaptation-steps value="5" />\n')
        f.write('        <device.rerouting.adaptation-interval value="180" />\n')
        f.write('        <device.rerouting.with-taz value="true" />\n')
        f.write('        <device.rerouting.threads value="5" />\n')
        f.write('        <device.rerouting.synchronize value="true" />\n')
        f.write('    </routing>\n')
    
        # Report settings
        f.write('    <report>\n')
        f.write('        <verbose value="true" />\n')
        f.write('        <xml-validation value="never" />\n')
        f.write('        <no-warnings value="true" />\n')
        f.write('        <error-log value="log.log" />\n')
        f.write('    </report>\n')
    
        # Mesoscopic simulation parameters
        f.write('    <mesoscopic>\n')
        f.write('        <mesosim value="true" />\n')
        f.write('        <meso-edgelength value="100" />\n')
        f.write('        <meso-tauff value="1.0" />\n')
        f.write('        <meso-taufj value="1.0" />\n')
        f.write('        <meso-taujf value="1.0" />\n')
        f.write('        <meso-taujj value="1.0" />\n')
        f.write('        <meso-jam-threshold value="-0.8" />\n')
        f.write('        <meso-multi-queue value="true" />\n')
        f.write('        <meso-junction-control value="true" />\n')
        f.write('        <meso-tls-penalty value="0" />\n')
        f.write('        <meso-tls-flow-penalty value="0" />\n')
        f.write('        <meso-minor-penalty value="0" />\n')
        f.write('        <meso-overtaking value="true" />\n')
        f.write('        <meso-lane-queue value="true" />\n')
        f.write('    </mesoscopic>\n')
    
        # Random number seed
        f.write('    <random_number>\n')
        f.write('        <seed value="17" />\n')
        f.write('    </random_number>\n')
    
        # GUI-specific settings (optional)
        f.write('    <gui_only>\n')
        f.write('        <tracker-interval value="2" />\n')
        f.write('    </gui_only>\n')
    
        # Close the configuration tag
        f.write('</configuration>\n')

    #print(f"SUMO configuration file '{config_file}' created successfully.")
    result = subprocess.run(["sumo", "-c", config_file], timeout=50)

    #print(f"SUMO simulation ran perfectly")

    
# Example usage:
for j in range(1,datasize):
  generate_random_simulations(j)  # Pass the TAZ file path
  print(f"done!{j}")

In [ ]:
#extracts edge_flows from routes file of the above generated SuMO simulation.

import xml.etree.ElementTree as ET
import pandas as pd
import torch

def count_edge_usage(routes_file, network_file):
    """
    Reads a routes file and network file, counts edge usage, and creates a table.

    Args:
        routes_file: Path to the routes file (XML).
        network_file: Path to the network file (NET.XML).

    Returns:
        A pandas DataFrame with edge IDs as columns and counts as values.
    """
    try:
        routes_tree = ET.parse(routes_file)
        routes_root = routes_tree.getroot()

        edge_ids = get_edge_ids(network_file)  # Assuming get_edge_ids is defined as in your previous code
        if not edge_ids:
            return None

        edge_counts = {edge_id: 0 for edge_id in edge_ids}

        for route in routes_root.findall('.//route'):
            edges = route.get('edges').split()
            for edge in edges:
                if edge in edge_counts:
                    edge_counts[edge] += 1

        #print(edge_counts)

        return edge_counts

    except FileNotFoundError:
        print(f"Error: File not found.")
        return None
    except ET.ParseError:
        print(f"Error: Invalid XML format.")
        return None
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return None

# Example usage:
y = []
routesdf = []
routes = []
for i in range(1,datasize):
    routes_file = "routes_WARB"+str(i)+".xml"  # Replace with your actual routes file path
    edge_usage = count_edge_usage(routes_file, network_file)
    y.append([edge_usage[edge] for edge in edge_list ])
    df = pd.DataFrame(list(edge_usage.items()), columns=['Edge', 'Count'])
    # dft = torch.tensor(df, dtype =torch.float)
    # print(dft)
    # y.append(dft)
    routesdf.append(df)
    routes.append(edge_usage)
    print(f"done!{i}")

y = torch.tensor(y, dtype=torch.float)
print(y.shape)

# # Display the DataFrame
# for i in range(len(routes)):
#    print(f"DataFrame {i+1}:")
#    print(routesdf[i])
# print(routes)

In [ ]:
#creates dataset for training GNN

import xml.etree.ElementTree as ET
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
!pip install torch_geometric
import torch_geometric
from torch_geometric.data import Data


def create_gnn_dataset(size):
    
    dataset = []
    for i in range(size):
        # Convert adjacency matrix to edge index
        edge_index = torch.tensor(np.nonzero(adjacency_matrix), dtype=torch.long)
        # Node_Features
        node_features = []
        for edges in edge_list:
              
            node_features.append([edge_coefficients[i][edges+str("in")]+edge_coefficients[i][edges+str("out")]])

        x = torch.tensor(node_features, dtype=torch.float)

        # Edge features (start and end destination)
        edge_attr = torch.zeros(len(incidence_matrix[0]))



        # Target variable (edge probabilities)
        # Mapping edge IDs to probabilities.  Need to handle potential missing keys.
        node_output = []
        for edges in edge_list:
            node_output.append([routes[i][edges]])

        y = torch.tensor(node_output, dtype=torch.float)

        data = Data(x=x, edge_index=edge_index, edge_attr=edge_attr,y=y)
        dataset.append(data)

    return dataset

# Create the GNN dataset
gnn_dataset = create_gnn_dataset(datasize-1)


print(gnn_dataset[9].edge_index.shape)
print(gnn_dataset[9].edge_attr.shape)
print(gnn_dataset[9].x.shape)
print(gnn_dataset[9].y.shape)